In [ ]:
from pathlib import Path
import datetime

import numpy as np
import polars as pl

import stock
from stock.trend_template import check_growing

In [ ]:
def get_simulation_results(
    watch_list: list[str], target_date: datetime.date = datetime.date.today()
):
    results = []
    for idx, code in enumerate(watch_list):
        stop_condition = stock.simulation.OnielStopCondition()
        res = stock.simulation.run(code, target_date, stop_condition)
        if res.buying_price > 0:
            results.append(
                {"code": code, "duration": res.duration, "profit": round(res.profit * 100)}
            )
    return results

In [ ]:
# 悪い決算を発表したが値上がりしてる銘柄を探す
current_date = datetime.date(year=2022, month=6, day=8)
code_list = stock.kabutan.get_code_list()

window_size = 2
growing_rate = 0.15
duration = 2

target_codes = []  # 直近の業績が悪かった銘柄リスト (売上は伸びてるけど利益は出てない)
target_dates = []
for code in code_list:
    df = stock.kabutan.read_financial_csv(stock.DATA_DIR / f"financial/{code}.csv")
    df = df.filter(pl.col("annoounce_date") <= current_date, pl.col("duration") == 3).sort(pl.col("annoounce_date"))
    if not check_growing(
            df,
            "total_revenue",
            growing_rate,
            min_duration=duration,
            current_date=current_date,
            num_average=window_size,
        ):
        continue


    if (
        check_growing(
            df,
            "operating_income",
            growing_rate,
            min_duration=duration,
            current_date=current_date,
            num_average=window_size,
        )
        or check_growing(
            df,
            "net_income",
            growing_rate,
            min_duration=duration,
            current_date=current_date,
            num_average=window_size,
        )
    ):
        continue
    target_codes.append(code)
    target_dates.append(df["annoounce_date"][-1] + datetime)

In [ ]:
# simulation結果を表示
sim_res = get_simulation_results(target_codes, target_date=current_date)
profits = np.array([res["profit"] for res in sim_res])
num_success = (profits >= 20).sum()
print("{} / {} ({:.2f} %)".format(num_success, len(profits), num_success / len(profits) * 100))